In [58]:
import pandas as pd
import numpy as np
import torch 
from torchvision import models, transforms
from PIL import Image
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import json
import tqdm


In [29]:
vgg16 = models.vgg16(pretrained=True)

/Users/davidgallardo/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/davidgallardo/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [31]:
vgg16.classifier = vgg16.classifier[:-1]

In [32]:
vgg16.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [33]:
def load_and_preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')

    # Resize while maintaining the original aspect ratio
    aspect_ratio = image.width / image.height
    if aspect_ratio > 1:
        target_size = (int(224 * aspect_ratio), 224)
    else:
        target_size = (224, int(224 / aspect_ratio))

    transform = transforms.Compose([
        transforms.Resize(target_size),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    input_image = transform(image).unsqueeze(0)
    return input_image

In [34]:
preprocessed_image = load_and_preprocess_image('images/2019_41005828_05.jpg')
preprocessed_image.shape

torch.Size([1, 3, 224, 224])

In [35]:
images_folder = 'images'

In [96]:
#warnings off pandas
pd.options.mode.chained_assignment = None

In [117]:
# Iterate over the images in the folder

image_features = pd.DataFrame()
count = 0
dict = {}
for filename in tqdm.tqdm(os.listdir(images_folder)):
    if filename.endswith(".jpg") or filename.endswith(".png"):

        count += 1
    
        # Load and preprocess the image
        image_path = os.path.join(images_folder, filename)
        input_image = load_and_preprocess_image(image_path)

        # Make predictions using the VGG16 model
        with torch.no_grad():
            output = vgg16(input_image)


        output = torch.nn.functional.normalize(output, p=2, dim=1)

        tensor_values = output.view(-1).tolist()

        # Create a dictionary containing the image filename and the tensor values
        dict[filename] = tensor_values

        # Append the dictionary to the image_features DataFrame, use concat



        #image_features = pd.concat([image_features, pd.DataFrame({'filename': filename, 'tensor_values': tensor_values})], ignore_index=True)
        


        # Print the top prediction
        #_, predicted_class = torch.max(output, 1)

        # Map the predicted class index to the label using the loaded JSON file
        #predicted_label = class_labels[predicted_class.item()]

        # Print the results
        #print(f"Image: {filename}, Predicted Class: {predicted_label} (Class Index: {predicted_class.item()})")

100%|██████████| 9248/9248 [14:05<00:00, 10.94it/s]


In [118]:
data = [{'Filename': filename, **{f'Weight_{i+1}': weight for i, weight in enumerate(weights)}}
        for filename, weights in dict.items()]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)

In [119]:
df.head()

,Filename,Weight_1,Weight_2,Weight_3,Weight_4,Weight_5,Weight_6,Weight_7,Weight_8,Weight_9,...,Weight_4087,Weight_4088,Weight_4089,Weight_4090,Weight_4091,Weight_4092,Weight_4093,Weight_4094,Weight_4095,Weight_4096
0,2023_57045979_TO.jpg,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.001199,0.0,...,0.035152,0.0,0.0,0.0,0.000000,0.008423,0.052571,0.0,0.000000,0.0
1,2023_57007758_92.jpg,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.028601,0.0,0.0,0.0,0.000000,0.000000,0.033525,0.0,0.000000,0.0
2,2023_57077728_76.jpg,0.0,0.0,0.009204,0.0,0.0,0.013492,0.000000,0.000000,0.0,...,0.005700,0.0,0.0,0.0,0.000000,0.038184,0.022136,0.0,0.000000,0.0
3,2023_57043798_99.jpg,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.070853,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.018857,0.0
4,2019_57017881_37.jpg,0.0,0.0,0.000000,0.0,0.0,0.000000,0.006811,0.000942,0.0,...,0.000000,0.0,0.0,0.0,0.007957,0.000000,0.015474,0.0,0.000000,0.0


In [120]:
df.shape

(9248, 4097)

In [122]:
import pickle

In [123]:
#save to a picle
df.to_pickle('image_features.pkl')

In [121]:
#save the df to json
df.to_json('image_features.json', orient='records')